In [6]:
!pip install gensim
!pip install pandas
!pip install numpy
!pip install nltk
!pip install scikit-learn

In [7]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [8]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

vking = wv['king']

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [10]:
wv.most_similar('man')

[('woman', 0.7664012908935547),
 ('boy', 0.6824871301651001),
 ('teenager', 0.6586930155754089),
 ('teenage_girl', 0.6147903203964233),
 ('girl', 0.5921714305877686),
 ('suspected_purse_snatcher', 0.571636438369751),
 ('robber', 0.5585119128227234),
 ('Robbery_suspect', 0.5584409832954407),
 ('teen_ager', 0.5549196600914001),
 ('men', 0.5489763021469116)]

In [14]:
wv.similarity('man','human')

np.float32(0.1473924)

In [16]:
import pandas as pd

In [17]:
messages = pd.read_csv('SMSSpamCollection.txt', sep='\t', names=['label','message'])

In [19]:
messages.shape
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [20]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
lm=WordNetLemmatizer()

In [24]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [25]:
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-z]',' ',messages['message'][i])
    review=review.lower()
    review=review.split()
    review=[lm.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [29]:
!pip install scikit-learn

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
cv = CountVectorizer(max_features=100, ngram_range=(1,2))
tfidf_transformer = TfidfTransformer()
clf = MultinomialNB()

In [35]:
X=cv.fit_transform(corpus).toarray()

In [ ]:
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000,
    formatter=dict(float=lambda x: "%.3g" % x))
X

In [37]:
cv.vocabulary_

{'go': np.int64(22),
 'great': np.int64(26),
 'got': np.int64(25),
 'wat': np.int64(89),
 'ok': np.int64(58),
 'free': np.int64(18),
 'win': np.int64(93),
 'text': np.int64(77),
 'txt': np.int64(85),
 'say': np.int64(68),
 'already': np.int64(0),
 'think': np.int64(80),
 'life': np.int64(38),
 'hey': np.int64(29),
 'week': np.int64(91),
 'back': np.int64(5),
 'like': np.int64(39),
 'still': np.int64(73),
 'send': np.int64(70),
 'friend': np.int64(19),
 'prize': np.int64(63),
 'claim': np.int64(9),
 'call': np.int64(6),
 'mobile': np.int64(49),
 'co': np.int64(10),
 'home': np.int64(31),
 'want': np.int64(88),
 'today': np.int64(82),
 'cash': np.int64(8),
 'day': np.int64(14),
 'reply': np.int64(65),
 'www': np.int64(96),
 'right': np.int64(66),
 'take': np.int64(75),
 'time': np.int64(81),
 'message': np.int64(46),
 'com': np.int64(11),
 'oh': np.int64(57),
 'yes': np.int64(99),
 'make': np.int64(44),
 'way': np.int64(90),
 'dont': np.int64(16),
 'miss': np.int64(48),
 'ur': np.int64(8

In [40]:
y=pd.get_dummies(messages['label'])
y=y.iloc[:,0].values
y.shape
y

array([ True,  True, False,  True,  True, False,  True,  True, False, False,  True, False, False,  True,  True, False,  True,  True,  True, False,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True, ...,  True,  True,  True,  True,  True, False,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True, False, False,  True,  True,  True,  True])

In [39]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [41]:
spam_detect_model=clf.fit(X_train,y_train)

In [42]:
y_pred=spam_detect_model.predict(X_test)

In [43]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [48]:
accuracy_score(y_test,y_pred)

0.9623318385650225

In [47]:

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.86      0.83      0.84       137
        True       0.98      0.98      0.98       978

    accuracy                           0.96      1115
   macro avg       0.92      0.91      0.91      1115
weighted avg       0.96      0.96      0.96      1115



In [51]:
  import nltk
  nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [58]:
[[i,j,k] for i,j,k in zip(list(map(len,corpus)),corpus, messages['message']) if i<1]

[[0, '', 'What you doing?how are you?'],
 [0, '', 'Where @'],
 [0, '', '645'],
 [0, '', 'Can a not?'],
 [0, '', ':) '],
 [0, '', 'What you doing?how are you?'],
 [0, '', ':( but your not here....'],
 [0, '', ':-) :-)']]

In [59]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess


In [60]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [61]:
model=gensim.models.Word2Vec(words)

In [ ]:
model.wv.index_to_key

In [63]:
model.corpus_count

5564

In [64]:
model.corpus_total_words

47567

In [65]:
model.epochs

5

In [66]:
model.wv.similar_by_word('good')

[('give', 0.9996167421340942),
 ('keep', 0.999586820602417),
 ('thing', 0.9995841979980469),
 ('like', 0.9995819330215454),
 ('back', 0.999578058719635),
 ('today', 0.9995648860931396),
 ('much', 0.9995638132095337),
 ('thanks', 0.9995574951171875),
 ('got', 0.9995511770248413),
 ('tell', 0.9995507597923279)]

In [67]:
model.wv['good'].shape

(100,)

In [68]:
words[0]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

In [74]:
def avg_word2vec(doc):
    word_vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if not word_vectors:  # If no words from the document are in the vocabulary
        return np.zeros(model.wv['good'].shape[0])  # Return a zero vector of the correct dimension (100)
    return np.mean(word_vectors, axis=0)

In [70]:
!pip install tqdm

In [71]:
from tqdm import tqdm

In [75]:
import numpy as np
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

100%|██████████| 5564/5564 [00:01<00:00, 3188.75it/s]


In [76]:
X_new=np.array(X)

In [77]:
y = messages[list(map(lambda x: len(x)>0 ,corpus))]
y=pd.get_dummies(y['label'])
y=y.iloc[:,0].values

In [78]:
X[0].reshape(1,-1).shape

(1, 100)

In [80]:
df = pd.DataFrame(X)

In [81]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [82]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [83]:
y_pred=classifier.predict(X_test)
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))

0.9703504043126685


In [86]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.91      0.84      0.87       135
        True       0.98      0.99      0.98       978

    accuracy                           0.97      1113
   macro avg       0.94      0.91      0.93      1113
weighted avg       0.97      0.97      0.97      1113

